In [1]:
import numpy as np
import pandas as pd
import datetime as dt

In [3]:
from pylab import mpl, plt
plt.style.use('seaborn')
mpl.rcParams['font.family'] = 'serif'
%matplotlib inline

In [4]:
import sys
sys.path.append('../dx')

In [5]:
'''
The Fundamental Theorem of Asset Pricing:
Martingale Measure: A probability measure that removes the drift associate 
with a Weiner process of the risk-free rate (i.e. makes the rate long sense stationary)

Example: Imagine a one day economy with a bond that costs 10 today and pays 10 tomorrow with P(1): E(x) = 0
and a stock that costs 10 and pays P(0.6) = 20 and P(0.4) = 0; E(x) = ((0.6*20) + (0.4*0)/10) - 1= 0.2

Now add an option pays P(0.6) = 5 and P(0.4) = 0; E(x) = (0.6*5) = 3

Another approach to this pricing is replicating the option through a portfolio of securities that replicates its
expected value. 
Buying 0.25 of the stock costs 2.5 (10 * .25) and has an expected value .25 * 20 = 5, so has the same expected value
of a win, but only costs (0.25 * 10) = 2.5. So the option is overvalued

Because of this, the seller of the option can fully hedge the risk by buying the portfolio that replicates the option
profile. The payoff of this hedged portfolio yields exactly the risk-free rate, because otherwise arbitrage would
result.

If we change the probabilities so the expected value is 0, the stock and options expected values become 0:
((0.5 * 20 + 0.5 * 0) / 10) - 1 = 0 is the expected return, and the option price is (0.5 * 5 + 0.5 * 0) = 2.5,
equivalent to the option-replicating portfolios cost.

'''

'\nThe Fundamental Theorem of Asset Pricing:\nMartingale Measure: A probability measure that removes the drift associate \nwith a Weiner process of the risk-free rate (i.e. makes the rate long sense stationary)\n\nExample: Imagine a one day economy with a bond that costs 10 today and pays 10 tomorrow with P(1): E(x) = 0\nand a stock that costs 10 and pays P(0.6) = 20 and P(0.4) = 0; E(x) = ((0.6*20) + (0.4*0)/10) - 1= 0.2\n\nNow add an option pays P(0.6) = 5 and P(0.4) = 0; E(x) = (0.6*5) = 3\n\nAnother approach to this pricing is replicating the option through a portfolio of securities that replicates its\nexpected value. \nBuying 0.25 of the stock costs 2.5 (10 * .25) and has an expected value .25 * 20 = 5, so has the same expected value\nof a win, but only costs (0.25 * 10) = 2.5. So the option is overvalued\n\nBecause of this, the seller of the option can fully hedge the risk by buying the portfolio that replicates the option\nprofile. The payoff of this hedged portfolio yields exa

In [6]:
'''
Let's generalize this to a discrete time market model M with parameters:
Sigma: a finite state space (i.e. discrete steps in the economies evolution)
F: A filtration
P: A probability measure defined on Sigma (in all states)
T: A terminal date (end of state space)
S: A set S[t][k] of security price processes (t=time, k=security)

Given this model, the following hold:
There are no arbitrage opportunities in M
The set Q of P-equivalent Martingale measures is not empty
The set P (diff from probability measure) of consistent linear price systems is nonempty

So:
if the market model M is arbitrage free, there exists a unique price V(0) associated with any derivative V(T). 
This satisfies the condition Q(V(0)) = Expected Value(e^-rT * V(t)). This is the risk-neutral valuation approach.

So our model will let all stochastic processes drift by the risk-free rate.
'''

"\nLet's generalize this to a discrete time market model M with parameters:\nSigma: a finite state space (i.e. discrete steps in the economies evolution)\nF: A filtration\nP: A probability measure defined on Sigma (in all states)\nT: A terminal date (end of state space)\nS: A set S[t][k] of security price processes (t=time, k=security)\n\nGiven this model, the following hold:\nThere are no arbitrage opportunities in M\nThe set Q of P-equivalent Martingale measures is not empty\nThe set P (diff from probability measure) of consistent linear price systems is nonempty\n\nSo:\nif the market model M is arbitrage free, there exists a unique price V(0) associated with any derivative V(T). \nThis satisfies the condition Q(V(0)) = Expected Value(e^-rT * V(t)). This is the risk-neutral valuation approach.\n\nSo our model will let all stochastic processes drift by the risk-free rate.\n"

In [9]:
'''
A derivatives analytics library has to discretize the time an option has until expiry, so it has a price at each
step. This can be done by counting days or using year fractions.

'''

'\nA derivatives analytics library has to discretize the time an option has until expiry, so it has a price at each\nstep. This can be done by counting days or using year fractions.\n\n'

In [10]:
dates = [dt.datetime(2020,1,1), dt.datetime(2020,7,1), dt.datetime(2021, 1, 1)]

In [12]:
(dates[1] - dates[0]).days/365

0.4986301369863014

In [13]:
(dates[2] - dates[1]).days/365

0.5041095890410959

In [14]:
fractions = [0.0, 0.5, 1]

In [19]:
# get year fractions out of a list of dates
def get_year_deltas(date_list, day_count=365):
    '''
    date_list: list or array - collection of datetime objects
    
    day_count: float - number of days in a year
    
    Results
    =========
    delta_list: array - year fractions
    '''
    start = date_list[0]
    delta_list = [(date - start).days / day_count for date in date_list]
    return np.array(delta_list)

In [20]:
get_year_deltas(dates)

array([0.        , 0.49863014, 1.00273973])

In [31]:
'''
This is useful for calculating the general discount factor, in this case
exp(-r * t) at time t
Modeling this as a class:
'''
class constant_short_rate(object):
    '''
    Class for constant short rate discounting.
    
    Attributes
    ===========
    name: string - name of the object
    short_rate: float (positive) - constant rate for discounting
    
    Results
    ==========
    get_discount_factors: get discount factors given a list/array of timestamp objects
    '''
    
    def __init__(self, name, short_rate):
        self.name = name
        self.short_rate = short_rate
        if(short_rate < 0):
            raise ValueError('Short rate negative.')
    
    def get_discount_factors(self, date_list, dtobjects=True):
        if dtobjects is True:
            dlist = get_year_deltas(date_list)
        else:
            dlist = np.array(date_list)
        dflist = np.exp(self.short_rate * np.sort(-dlist)) # take negative of time fractions
        return np.array((date_list, dflist)).T

In [32]:
csr = constant_short_rate('csr', 0.05)

In [33]:
csr.get_discount_factors(dates)

array([[datetime.datetime(2020, 1, 1, 0, 0), 0.9510991280247174],
       [datetime.datetime(2020, 7, 1, 0, 0), 0.9753767163648953],
       [datetime.datetime(2021, 1, 1, 0, 0), 1.0]], dtype=object)

In [35]:
deltas = get_year_deltas(dates)
deltas

array([0.        , 0.49863014, 1.00273973])

In [37]:
csr.get_discount_factors(deltas, dtobjects=False)

array([[0.        , 0.95109913],
       [0.49863014, 0.97537672],
       [1.00273973, 1.        ]])

In [41]:
class market_environment(object):
    '''
    class to model a market environment relevant for valuation
    
    Attributes
    ===========
    name: string - name of the market environment
    pricing_date: datetime - date of the market environment
    
    Methods
    =========
    add_constant: adds a constant market parameter
    get_constant: get constant
    add_lists: adds a list (of underlyings)
    get_list: gets a list
    add_curve: adds a market curve (i.e. yield curve)
    get_curve: gets a market curve
    add_environment: upserts whole market environments with constants, lists, curves
    
    '''
    def __init__(self, name, pricing_date):
        self.name = name
        self.pricing_date = pricing_date
        self.constants = {}
        self.lists = {}
        self.curves = {}
        
    def add_constant(self, key, constant):
        self.constants[key] = constant
    
    def get_constant(self, key):
        return self.constants[key]
    
    def add_list(self, key, list_object):
        self.lists[key] = list_object
        
    def get_list(self, key):
        return self.lists[key]
    
    def add_curve(self, key, curve):
        self.curves[key] = curve
        
    def get_curve(self, key):
        return self.curves[key]
    
    def add_environment(self, env):
        # overwrite values for class if they exist
        self.constants.update(env.constants)
        self.lists.update(env.lists)
        self.curves.update(env.curves)
        

In [42]:
me = market_environment('me_gbm', dt.datetime(2020, 1, 1))

In [43]:
me.add_constant('initial_value', 36.)

In [44]:
me.add_constant('volatility', 0.2)

In [45]:
me.add_constant('final_date', dt.datetime(2020, 12, 31))

In [46]:
me.add_constant('currency', 'EUR')

In [47]:
me.add_constant('frequency', 'M')

In [48]:
me.add_constant('paths', 10000)

In [49]:
me.add_curve('discount_curve', csr)

In [50]:
me.get_constant('volatility')

0.2

In [52]:
me.get_curve('discount_curve').short_rate

0.05